In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
def Rij(gene_length):
    return [[random.uniform(0.5, 1.0) for _ in range(gene_length)] 
            for _ in range(gene_length)]

In [ ]:
def fitness_function(genotype, Rij_matrix, gene_length):
    gene_i = genotype[:gene_length].count("1")
    gene_j = genotype[gene_length:].count("1")
    fitness = Rij_matrix[gene_i - 1][gene_j - 1] * ((2 ** gene_i) + (2 ** gene_j))
    return fitness

In [ ]:
def check_population_fitness(population, Rij_matrix, gene_length):
    fitness_values = [fitness_function(genotype, Rij_matrix, gene_length) for genotype in population]
    return fitness_values

In [ ]:
def genotype_generator(gene_length):
    genotype_length = gene_length * 2
    genotype = "".join(random.choice("01") for _ in range(genotype_length))
    return genotype

In [ ]:
def generate_population(population_size, gene_length):
    population = [genotype_generator(gene_length) for _ in range(population_size)]
    return population

In [ ]:
def roulette_wheel_selection(population, fitness_values):
    total_fitness = 0
    accumulated_fitness = []
    for fitness in fitness_values:
        total_fitness += fitness
        accumulated_fitness.append(total_fitness)
        
    random_val = random.uniform(0, total_fitness)
    for i in range(len(accumulated_fitness)):
        if random_val <= accumulated_fitness[i]:
            return population[i]
     

In [ ]:
def mutation(genotype, mutation_rate):
    genotype_list = list(genotype)
    for i in range(len(genotype_list)):
        if random.random() <= mutation_rate:
            genotype_list[i] = '1' if genotype_list[i] == '0' else '0'
    return ''.join(genotype_list)

In [ ]:
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    return child1

In [ ]:
def run_simulation_mutation_only(gene_length, amount_of_demes, population_per_deme, mutation_rate, log=False):
    simulations_to_run = 30
    simulations = []
    for x in range(simulations_to_run):
        R_noise = Rij(gene_length)
        generations = 0
        demes = [generate_population(population_per_deme, gene_length) for _ in range(amount_of_demes)]
        best_possible_fitness = fitness_function("1" * (2 * gene_length), R_noise, gene_length)
        max_fitness_found_in_all_generations = 0
        while (max_fitness_found_in_all_generations != best_possible_fitness) and (generations < 2000):
            generations += 1
            if log:
                print(f"Simulations Left: {simulations_to_run - x} || Generation:  {generations}")
            demes_fitnesses = [check_population_fitness(deme, R_noise, gene_length) for deme in demes]
            demes_best_fitness_index = [deme_fitness.index(max(deme_fitness)) for deme_fitness in demes_fitnesses]
            dene_elitism_individuals = [demes[i][demes_best_fitness_index[i]] for i in range(amount_of_demes)]
            new_demes = [[fittest_individual] for fittest_individual in dene_elitism_individuals]
            
            # Migration
            for deme_index, deme in enumerate(demes):
                deme_to_take_migrant_from = random.choice([i for i in range(amount_of_demes) if i != deme_index])
                migrant = random.choice(demes[deme_to_take_migrant_from])
                new_demes[deme_index].append(migrant)
            
            for i in range(amount_of_demes):
                deme_to_mutate = demes[i]
                deme_fitness = demes_fitnesses[i]
                new_deme = new_demes[i]
                while len(new_deme) < population_per_deme:
                    individual_to_be_mutated = roulette_wheel_selection(deme_to_mutate, deme_fitness)
                    new_individual = mutation(individual_to_be_mutated, mutation_rate)
                    new_deme.append(new_individual)
            demes = new_demes
            demes_fitnesses = [check_population_fitness(deme, R_noise, gene_length) for deme in demes]
            best_fitness_this_generation = max([max(deme_fitness) for deme_fitness in demes_fitnesses])
            max_fitness_found_in_all_generations = max(max_fitness_found_in_all_generations, best_fitness_this_generation)
        if generations < 2000:
            simulations.append(generations)
    return simulations
    

In [ ]:
def run_simulation_crossover_and_mutation(gene_length, amount_of_demes, population_per_deme, mutation_rate, log=False):
    simulations_to_run = 30
    simulations = []
    for x in range(simulations_to_run):
        R_noise = Rij(gene_length)
        generations = 0
        demes = [generate_population(population_per_deme, gene_length) for _ in range(amount_of_demes)]
        best_possible_fitness = fitness_function("1" * (2 * gene_length), R_noise, gene_length)
        max_fitness_found_in_all_generations = 0
        while (max_fitness_found_in_all_generations != best_possible_fitness) and (generations < 2000):
            generations += 1
            if log:
                print(f"Simulations Left: {simulations_to_run - x} || Generation:  {generations}")
            demes_fitnesses = [check_population_fitness(deme, R_noise, gene_length) for deme in demes]
            demes_best_fitness_index = [deme_fitness.index(max(deme_fitness)) for deme_fitness in demes_fitnesses]
            dene_elitism_individuals = [demes[i][demes_best_fitness_index[i]] for i in range(amount_of_demes)]
            new_demes = [[fittest_individual] for fittest_individual in dene_elitism_individuals]
            
            # Migration
            for deme_index, deme in enumerate(demes):
                deme_to_take_migrant_from = random.choice([i for i in range(amount_of_demes) if i != deme_index])
                migrant = random.choice(demes[deme_to_take_migrant_from])
                new_demes[deme_index].append(migrant)
            
            for i in range(amount_of_demes):
                deme_to_mutate = demes[i]
                deme_fitness = demes_fitnesses[i]
                new_deme = new_demes[i]
                while len(new_deme) < population_per_deme:
                    parent1 = roulette_wheel_selection(deme_to_mutate, deme_fitness)
                    parent2 = roulette_wheel_selection(deme_to_mutate, deme_fitness)
                    offspring = crossover(parent1, parent2)
                    offspring = mutation(offspring, mutation_rate)
                    new_deme.append(offspring)
            demes = new_demes
            demes_fitnesses = [check_population_fitness(deme, R_noise, gene_length) for deme in demes]
            best_fitness_this_generation = max([max(deme_fitness) for deme_fitness in demes_fitnesses])
            max_fitness_found_in_all_generations = max(max_fitness_found_in_all_generations, best_fitness_this_generation)
        if generations < 2000:
            simulations.append(generations)
    
    return simulations
    

In [ ]:
mutation_only_gene_length = [10,20,30,40, 50]

total_population = 400
amount_of_demes = 20
population_per_deme = total_population // amount_of_demes

mutation_only_results = {}
for n in mutation_only_gene_length:
    print(f"Mutation_Only: Running mutation only simulation for gene length {n}")
    mutation_rate = 1 / (2 * n)
    results = run_simulation_mutation_only(n, amount_of_demes, population_per_deme, mutation_rate)
    print(f"Mutation_Only: Results for gene length {n}: {results}")
    if results != []:
        mean = np.mean(results)
        std_dev = np.std(results)
        mutation_only_results[n] = {
            "mean": mean,
            "std": std_dev
        }
        print(f"Mutation_Only: Results for gene length {n}: {mutation_only_results[n]}")
    else:
        print(f"Mutation_Only: Results for gene length {n}: COULDNT FIND SOLUTION!")



In [ ]:
crossover_and_mutation_gene_length = [10,20,30,40,50, 60, 70, 80]
crossover_mutation_results = {}
for n in crossover_and_mutation_gene_length:
    print(f"Crossover_Mutation: Running crossover and mutation simulation for gene length {n}")
    mutation_rate = 1 / (2 * n)
    results = run_simulation_crossover_and_mutation(n, amount_of_demes, population_per_deme, mutation_rate)
    print(f"Crossover_Mutation: Results for gene length {n}: {results}")
    if results != []:
        mean = np.mean(results)
        std_dev = np.std(results)
        crossover_mutation_results[n] = {
            "mean": mean,
            "std": std_dev
        }
    else:
        print(f"Crossover_Mutation: Results for gene length {n}: COULDNT FIND SOLUTION!")
        
with open('reimplement.txt', 'w') as f:
    json.dump(crossover_mutation_results, f, indent=4)

In [ ]:
mutation_n_values = sorted(mutation_only_results.keys())
mutation_means = [mutation_only_results[n]['mean'] for n in mutation_n_values]
mutation_stds = [mutation_only_results[n]['std'] for n in mutation_n_values]

crossover_n_values = sorted(crossover_mutation_results.keys())
crossover_means = [crossover_mutation_results[n]['mean'] for n in crossover_n_values]
crossover_stds = [crossover_mutation_results[n]['std'] for n in crossover_n_values]

plt.figure(figsize=(10, 6)) 
plt.plot(mutation_n_values, mutation_means, 
             marker='o', linestyle='-', color='red', linewidth=2, 
             markersize=6, label='no crossover')

for i, (x, y, std) in enumerate(zip(mutation_n_values, mutation_means, mutation_stds)):
        plt.plot([x, x], [y-std, y+std], color='red', linewidth=1)
        plt.plot([x-0.5, x+0.5], [y-std, y-std], color='red', linewidth=1)
        plt.plot([x-0.5, x+0.5], [y+std, y+std], color='red', linewidth=1)

plt.plot(crossover_n_values, crossover_means, 
             marker='s', linestyle='-', color='blue', linewidth=2, 
             markersize=6, label='with crossover')

for i, (x, y, std) in enumerate(zip(crossover_n_values, crossover_means, crossover_stds)):
        plt.plot([x, x], [y-std, y+std], color='blue', linewidth=1)
        plt.plot([x-0.5, x+0.5], [y-std, y-std], color='blue', linewidth=1)
        plt.plot([x-0.5, x+0.5], [y+std, y+std], color='blue', linewidth=1)

plt.xlabel('n', fontsize=12)
plt.ylabel('generation to peak', fontsize=12)
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 90)
plt.ylim(0, 2000)
plt.xticks(range(0, 91, 10))
plt.yticks(range(0, 2001, 200))

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6)) 
plt.semilogy(mutation_n_values, mutation_means, 
         marker='o', linestyle='-', color='red', linewidth=2, 
         markersize=6, label='no crossover')

for i, (x, y, std) in enumerate(zip(mutation_n_values, mutation_means, mutation_stds)):
    plt.plot([x, x], [y-std, y+std], color='red', linewidth=1)
    plt.plot([x-0.5, x+0.5], [y-std, y-std], color='red', linewidth=1)
    plt.plot([x-0.5, x+0.5], [y+std, y+std], color='red', linewidth=1)

plt.semilogy(crossover_n_values, crossover_means, 
         marker='s', linestyle='-', color='blue', linewidth=2, 
         markersize=6, label='with crossover')

for i, (x, y, std) in enumerate(zip(crossover_n_values, crossover_means, crossover_stds)):
    plt.plot([x, x], [y-std, y+std], color='blue', linewidth=1)
    plt.plot([x-0.5, x+0.5], [y-std, y-std], color='blue', linewidth=1)
    plt.plot([x-0.5, x+0.5], [y+std, y+std], color='blue', linewidth=1)

plt.xlabel('n', fontsize=12)
plt.ylabel('generation to peak', fontsize=12)
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 90)
plt.ylim(1, 10000)
plt.xticks(range(0, 91, 10))

plt.tight_layout()
plt.show()